<a href="https://colab.research.google.com/gist/cdmafAdi/758818e4e0029afc2aee1429020c3d9c/eco_friendly_multi_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install replicate gradio duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.6 MB/s eta 0:00:00


In [2]:
import os
import replicate
import gradio as gr
from duckduckgo_search import DDGS

# 🔐 Replicate API Token (Recommended: store in .env or input box)
REPLICATE_API_TOKEN = "r8_BQu9uxNcWbj0xmlQPrqLoWUyRcsxva61K67VZ"  # Replace with your token
client = replicate.Client(api_token=REPLICATE_API_TOKEN)

# 🧠 IBM Granite Model ID
GRANITE_MODEL = "ibm-granite/granite-3.3-8b-instruct"

# 🔎 RAG Search via DuckDuckGo (trusted sources)
def search_duckduckgo(query):
    try:
        with DDGS() as ddgs:
            results = ddgs.text(
                f"{query} site:gov OR site:org OR site:nationalgeographic.com OR site:carbontrust.com OR site:greenpeace.org",
                max_results=3
            )
            return "\n".join([f"- {r['title']}: {r['href']}" for r in results])
    except Exception as e:
        return "- No external sources found or error during search."

# 🤖 Call IBM Granite via Replicate
def query_replicate(prompt):
    try:
        output = client.run(GRANITE_MODEL, input={"prompt": prompt, "max_new_tokens": 350})
        return "".join(output)
    except Exception as e:
        return f"❌ Error from Replicate: {str(e)}"

# 🌱 Agent 1: Eco Lifestyle Agent
def eco_lifestyle_agent(user_prompt, location, category):
    search_query = f"{user_prompt} {category} in {location}" if location else f"{user_prompt} {category}"
    sources = search_duckduckgo(search_query)
    if not sources.strip():
        sources = "- No sources found. Providing general sustainability practices."

    system_prompt = (
        f"You are Eco Lifestyle AI, helping users adopt a greener lifestyle.\n"
        f"Focus: personalized sustainable living advice, eco tips, and awareness.\n"
        f"User Location: {location or 'unspecified'} | Topic: {category}\n"
        f"User Query: {user_prompt}\n\nSources:\n{sources}\n\nAnswer:"
    )
    return query_replicate(system_prompt)

# 🛍️ Agent 2: Eco Product Finder
def eco_product_agent(user_prompt, location):
    search_query = f"{user_prompt} eco-friendly product {location}"
    sources = search_duckduckgo(search_query)
    if not sources.strip():
        sources = "- No product results found. Giving general eco product advice."

    system_prompt = (
        f"You are Eco Product Finder. Recommend eco-friendly product alternatives.\n"
        f"Location: {location or 'global'} | Query: {user_prompt}\n"
        f"Sources:\n{sources}\n\nSuggest green, sustainable, ethical alternatives with brief explanation."
    )
    return query_replicate(system_prompt)

# 🔄 Agent 3: Recycling Rules Bot
def recycling_rules_agent(user_prompt, location):
    search_query = f"{user_prompt} recycling rules in {location}"
    sources = search_duckduckgo(search_query)
    if not sources.strip():
        sources = "- No local rules found. Sharing general recycling guidelines."

    system_prompt = (
        f"You are Recycling Rules Bot. Help users understand recycling rules.\n"
        f"Location: {location or 'not specified'} | Query: {user_prompt}\n"
        f"Sources:\n{sources}\n\nAnswer:"
    )
    return query_replicate(system_prompt)

# 🎨 Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 🌿 Multi-Agent Eco Assistant")
    gr.Markdown("Powered by IBM Granite + RAG | Get green lifestyle tips, product suggestions, and recycling help.")

    with gr.Tabs():
        with gr.Tab("🌱 Eco Lifestyle Agent"):
            q1 = gr.Textbox(label="Ask about green lifestyle")
            l1 = gr.Textbox(label="Your location", placeholder="e.g., Mumbai")
            c1 = gr.Dropdown(choices=["Home", "Travel", "Shopping", "Recycling", "Energy", "Diet", "General"], value="General", label="Topic")
            out1 = gr.Textbox(label="Response")
            btn1 = gr.Button("Get Advice")
            btn1.click(eco_lifestyle_agent, inputs=[q1, l1, c1], outputs=out1)

        with gr.Tab("🛍️ Eco Product Finder"):
            q2 = gr.Textbox(label="Product query", placeholder="e.g., eco-friendly shampoo")
            l2 = gr.Textbox(label="Your location", placeholder="e.g., India")
            out2 = gr.Textbox(label="Product Suggestions")
            btn2 = gr.Button("Find Products")
            btn2.click(eco_product_agent, inputs=[q2, l2], outputs=out2)

        with gr.Tab("🔄 Recycling Rules Bot"):
            q3 = gr.Textbox(label="Recycling question", placeholder="e.g., Can I recycle bubble wrap?")
            l3 = gr.Textbox(label="Your location", placeholder="e.g., Delhi")
            out3 = gr.Textbox(label="Recycling Guidance")
            btn3 = gr.Button("Check Rules")
            btn3.click(recycling_rules_agent, inputs=[q3, l3], outputs=out3)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9d055f576b30142e3a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
